In [4]:
import pulp

In [5]:
model_4_a = pulp.LpProblem("Multinamtional_company", pulp.LpMinimize)
Production_center = ["A", "B", "C", "D"]
Warehouse = [1, 2, 3]
Distribution_center = [1, 2, 3, 4, 5, 6]
X_1 = pulp.LpVariable.dicts(f"ProductionCenter_to_Warehouse", [(i,j) for i in Production_center for j in Warehouse], lowBound=0, cat=pulp.LpInteger)
X_2 = pulp.LpVariable.dicts(f"Warehouse_to_DistributionCenter", [(i,j) for i in Warehouse for j in Distribution_center], lowBound=0, cat=pulp.LpInteger)
Costs_fromProdctionToWarehouse_matrix=[
    [4, 3, 5],
    [6, 2, 4],
    [5, 4, 3],
    [7, 5, 6]
]
fist_half_route = [
    [X_1["A", i] for i in Warehouse],
    [X_1["B", i] for i in Warehouse],
    [X_1["C", i] for i in Warehouse],
    [X_1["D", i] for i in Warehouse],
]
Costs_fromProdctionToWarehouse = pulp.lpDot(Costs_fromProdctionToWarehouse_matrix, fist_half_route)
Costs_fromWarehouseToDistributionCenter_matrix=[
    [3, 2, 5, 4, 6, 7],
    [1, 4, 2, 3, 5, 6],
    [5, 3, 4, 2, 3, 4]
]
second_half_route = [
    [X_2[1, i] for i in Distribution_center],
    [X_2[2, i] for i in Distribution_center],
    [X_2[3, i] for i in Distribution_center],
]
Costs_fromWarehouseToDistributionCenter = pulp.lpDot(Costs_fromWarehouseToDistributionCenter_matrix, second_half_route)
Costs_fromWarehouseToDistributionCenter

model_4_a += Costs_fromProdctionToWarehouse + Costs_fromWarehouseToDistributionCenter, "Objective"

# flow in and out constriants
for i in Warehouse:
    model_4_a += pulp.lpSum(X_1[in_, i] for in_ in Production_center) == pulp.lpSum(X_2[i, out_] for out_ in Distribution_center), f"Equal_in_and_out_flow_ofWarehouse{i}"

# supply and demand cap
Prodiction_supply = [70, 90, 80, 60]
for index_ in range(len(Production_center)):
    model_4_a += pulp.lpSum(X_1[Production_center[index_], j] for j in Warehouse) == Prodiction_supply[index_], f"Supply_from_ProductionCenter{Production_center[index_]}"

Distribution_demand = [50, 40, 60, 70, 50, 30]
for index_ in range(len(Distribution_center)):
    model_4_a += pulp.lpSum(X_2[j, Distribution_center[index_]] for j in Warehouse) == Distribution_demand[index_], f"Demand_from_distributionCenter{Distribution_center[index_]}"

# To be accordance with Excel results
model_4_a += X_2[2,4] == 10, "Accordance_with_Excel"
model_4_a.solve()
print("Model Status:{}".format(pulp.LpStatus[model_4_a.status]))
for v in model_4_a.variables():
	  print(v.name, "=", v.varValue, "\tReduced Cost =", v.dj)
print("Objective=", pulp.value(model_4_a.objective))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/yanhanjun/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/8t/3xrvdszj1g75llc2jb40dnvw0000gn/T/fe21a8dc735f4467ad40f93670f6e55f-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/8t/3xrvdszj1g75llc2jb40dnvw0000gn/T/fe21a8dc735f4467ad40f93670f6e55f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 19 COLUMNS
At line 171 RHS
At line 186 BOUNDS
At line 217 ENDATA
Problem MODEL has 14 rows, 30 columns and 61 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1700 - 0.00 seconds
Cgl0004I processed model has 12 rows, 28 columns (28 integer (0 of which binary)) and 56 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I Integer solution of 1700 found by DiveCoefficient after 0 iterations and 0 nodes (0.02 seconds)
Cbc

In [6]:
X_2[2,4]

Warehouse_to_DistributionCenter_(2,_4)